In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Pipeline at a glance:
load data -> cleaning/preprocessing -> feature eng -> format data for modelling -> fit model -> evaluate model -> generate submission file

## Step 1: Load data

In [2]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')

# for convenience
datasets = [df_train, df_test]

## Step 2: Basic cleaning and preprocessing

In [3]:
# Fill in missing values
medFare = pd.concat((df_train['Fare'], df_test['Fare']), axis=0).median()
for df in datasets:
    df['Fare'] = df['Fare'].fillna(medFare)
    df['Embarked'] = df['Embarked'].fillna('S')

## Step 3: Feature engineering

In [4]:
# Functions that returns the title from a name. All the name in the dataset has the format "Surname, Title. Name"
def get_title(name):
    if '.' in name:
        return name.split(',')[1].split('.')[0].strip()
    else:
        return 'Unknown'
    
train_titles = []
test_titles = []

for name in df_train['Name']:
    title = get_title(name)
    train_titles.append(title)
    
for name in df_test['Name']:
    title = get_title(name)
    test_titles.append(title)


df_train['Title'] = train_titles
df_test['Title'] = test_titles
     
df_train.loc[(df_train['Title'] != 'Mr') & (df_train['Title'] != 'Mrs') & (df_train['Title'] != 'Miss') & (df_train['Title'] != 'Master'), 'Title'] = 'Misc'
df_test.loc[(df_test['Title'] != 'Mr') & (df_test['Title'] != 'Mrs') & (df_test['Title'] != 'Miss') & (df_test['Title'] != 'Master'), 'Title'] = 'Misc'

# See value counts of each title
pd.value_counts(df_train['Title'])


Mr        517
Miss      182
Mrs       125
Master     40
Misc       27
Name: Title, dtype: int64

In [5]:
#thing = (pd.concat((df_train, df_test), ignore_index=True)).groupby(['Title']).mean()
#print(thing)

In [6]:
grouped = (pd.concat((df_train, df_test), ignore_index=True)).groupby(['Title'])  

# opted for median value to iron out any weirdness title selection might've given
grouped.Age.median()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


Title
Master     4.0
Misc      44.5
Miss      22.0
Mr        29.0
Mrs       35.5
Name: Age, dtype: float64

In [7]:
# Apply these median values to all NaNs in dataset
for df in datasets:
    df.Age = grouped.Age.apply(lambda x: x.fillna(x.median()))

# Check that baby out
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr
5,6,0,3,"Moran, Mr. James",male,29.0,0,0,330877,8.4583,NaN,Q,Mr
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,Mr
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,Master
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,Mrs
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,Mrs


In [8]:
# find fare bins based on frequency rather than value
garbage, fare_bins = pd.qcut(df_train['Fare'].append(df_test['Fare']), 4, retbins=True)
print(fare_bins)
# loops through both train and test set for convenience
for df in datasets:
    # bin age
    df['Age_binned'] = pd.cut(df['Age'], [0,16,32,48,64,200], labels = [0,1,2,3,4], retbins=False)
    
    # bin fare
    df['Fare_binned'] = pd.cut(df['Fare'], fare_bins, labels = [0,1,2,3], include_lowest=True, retbins=False)
    
    # family features
    df['Family_size'] = df_train['SibSp'] + df_train['Parch']
    df['Is_Alone'] = (df['Family_size'] == 0).astype(int)

[  0.       7.8958  14.4542  31.275  512.3292]


In [9]:
# drop unneeded rows
for df in datasets:
    df.drop(['Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Name'], axis=1, inplace=True)
    
# need to keep passengerID for submission

## Step 4: Format data for modelling

In [10]:
# one-hot encode categoricals using pd.get_dummies
df_train = pd.get_dummies(df_train, columns=['Embarked', 'Sex', 'Title'])
df_test = pd.get_dummies(df_test, columns=['Embarked', 'Sex', 'Title'])
df_train.head()

,PassengerId,Survived,Pclass,Age_binned,Fare_binned,Family_size,Is_Alone,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,Title_Master,Title_Misc,Title_Miss,Title_Mr,Title_Mrs
0,1,0,3,1,0,1,0,0,0,1,0,1,0,0,0,1,0
1,2,1,1,2,3,1,0,1,0,0,1,0,0,0,0,0,1
2,3,1,3,1,1,0,1,0,0,1,1,0,0,0,1,0,0
3,4,1,1,2,3,1,0,0,0,1,1,0,0,0,0,0,1
4,5,0,3,2,1,0,1,0,0,1,0,1,0,0,0,1,0


In [11]:
# Split data into X and y, and select features to use
X = df_train.drop(['Survived', 'PassengerId'], axis=1)
y = df_train['Survived']
X_test = df_test.drop(['PassengerId'], axis=1)

In [12]:
# Split data X and y into train and val sets
X_train, X_val, y_train, y_val = train_test_split(
      X, y, test_size=0.2, random_state=42)

In [13]:
df_train.isnull().sum()

PassengerId     0
Survived        0
Pclass          0
Age_binned      0
Fare_binned     0
Family_size     0
Is_Alone        0
Embarked_C      0
Embarked_Q      0
Embarked_S      0
Sex_female      0
Sex_male        0
Title_Master    0
Title_Misc      0
Title_Miss      0
Title_Mr        0
Title_Mrs       0
dtype: int64

In [14]:
df_test.isnull().sum()

PassengerId     0
Pclass          0
Age_binned      0
Fare_binned     0
Family_size     0
Is_Alone        0
Embarked_C      0
Embarked_Q      0
Embarked_S      0
Sex_female      0
Sex_male        0
Title_Master    0
Title_Misc      0
Title_Miss      0
Title_Mr        0
Title_Mrs       0
dtype: int64

In [15]:
X_train

,Pclass,Age_binned,Fare_binned,Family_size,Is_Alone,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,Title_Master,Title_Misc,Title_Miss,Title_Mr,Title_Mrs
331,1,2,2,0,1,0,0,1,0,1,0,0,0,1,0
733,2,1,1,0,1,0,0,1,0,1,0,0,0,1,0
382,3,1,1,0,1,0,0,1,0,1,0,0,0,1,0
704,3,1,0,1,0,0,0,1,0,1,0,0,0,1,0
813,3,0,2,6,0,0,0,1,1,0,0,0,1,0,0
118,1,1,3,1,0,1,0,0,0,1,0,0,0,1,0
536,1,2,2,0,1,0,0,1,0,1,0,1,0,0,0
361,2,1,2,1,0,1,0,0,0,1,0,0,0,1,0
29,3,1,0,0,1,0,0,1,0,1,0,0,0,1,0
55,1,1,3,0,1,0,0,1,0,1,0,0,0,1,0


## Step 5: Train (or fit) the model

In [16]:
# Fit logistic regression
lr = LogisticRegression()
lr.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [17]:
# fit rf
rf = RandomForestClassifier(random_state=12)
rf.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=12, verbose=0, warm_start=False)

## Step 6: Evaluate the model

In [18]:
# define a scoring function
def acc(y: np.array, y_pred: np.array) -> float:
    return np.sum(y_pred==y)/len(y)

In [19]:
# Use model to predict on train and val sets
y_pred_trn_lr = lr.predict(X_train)
y_pred_val_lr = lr.predict(X_val)

y_pred_trn_rf = rf.predict(X_train)
y_pred_val_rf = rf.predict(X_val)

In [20]:
print('LR train set accuracy', acc(y_train, y_pred_trn_lr))
print('LR val set accuracy', acc(y_val, y_pred_val_lr))
print('RF train set accuracy', acc(y_train, y_pred_trn_rf))
print('RF val set accuracy', acc(y_val, y_pred_val_rf))

LR train set accuracy 0.8328651685393258
LR val set accuracy 0.8044692737430168
RF train set accuracy 0.8876404494382022
RF val set accuracy 0.8379888268156425


## Step 7: Generate Submission

In [21]:
# Predict for test set

y_pred_test = rf.predict(X_test)

# Create a Kaggle submission 
sub = pd.DataFrame({'PassengerId': df_test['PassengerId'],
                    'Survived': y_pred_test})

sub.to_csv('week_3_baseline-finished.csv', index=False)